# K Nearest Neighbours Classifier
In this notebook, I will be creating a K Nearest Neighbours clasifier for the loan data.

Make a simple model that can be used as the baseline to test against
List of improvements that could be made:
* Scaling data (using Z scores) to improve perforamnce. This is because if some features are too large, they might "dominate the distance calculation and distort the cluster formation"
* Dimensionality reduction (pca/lda/t-sne/autoencoder/multidimensional scaling etc)due to the curse of dimensionality that knn has
* Selection of distance function
* Selection of K


In [22]:
import pandas as pd
import numpy as np
import sklearn

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import RadiusNeighborsClassifier

from sklearn import metrics 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score

from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import GridSearchCV

In [23]:
pd.set_option('display.max_columns', None)

loan_data = pd.read_pickle("my_full_data.pkl")
display(loan_data)

,id,loan_amnt,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,addr_state,dti,open_acc,total_acc,initial_list_status,last_fico_range_high,last_fico_range_low,application_type,tax_liens
0,68387134,2500.0,0,17.27,89.47,3,1,1,50000.0,2,1,9,9,17.60,7.0,22.0,1,669.0,665.0,0,0.0
1,66615254,7200.0,0,17.27,257.67,3,9,4,20000.0,2,0,2,18,13.21,8.0,17.0,1,549.0,545.0,0,0.0
2,68376899,16000.0,0,10.78,522.16,1,5,5,63000.0,0,0,2,33,35.19,12.0,22.0,1,594.0,590.0,0,0.0
3,68366850,16000.0,0,5.32,481.84,0,8,1,96000.0,0,1,2,22,14.45,23.0,38.0,1,744.0,740.0,0,0.0
4,67195202,13000.0,1,16.59,320.23,3,10,5,40000.0,2,0,2,11,16.74,6.0,17.0,1,569.0,565.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26113,90441274,12000.0,0,13.99,410.08,2,0,5,47000.0,1,1,1,25,31.79,13.0,28.0,0,754.0,750.0,0,0.0
26114,90404436,25000.0,1,25.69,743.94,5,6,1,70000.0,2,0,9,5,15.10,9.0,19.0,0,499.0,0.0,0,0.0
26115,90393111,16000.0,0,12.79,537.49,2,1,4,48000.0,0,1,2,4,33.60,11.0,17.0,0,699.0,695.0,0,0.0
26116,90304259,35000.0,1,15.59,843.53,2,1,1,180000.0,2,1,2,9,13.09,17.0,31.0,0,714.0,710.0,0,0.0


## Baseline Model

In [24]:
X = loan_data.drop(['loan_status', 'id'], axis=1) #note that donr was dropped from X because it is the dependent variable
y = loan_data['loan_status']

#Create training and test datasets
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size = 0.20, random_state = 5)

X_test_2=X_test
X_train_2=X_train
X_test_3=X_test
X_train_3=X_train

In [25]:
KNN_model_default = KNeighborsClassifier()
KNN_model_default.fit(X_train, y_train)
y_pred_KNN_default = KNN_model_default.predict(X_test)

In [26]:
print(classification_report(y_test, y_pred_KNN_default))
print('Accuracy Score - KNN - Default:', metrics.accuracy_score(y_test, y_pred_KNN_default))  

              precision    recall  f1-score   support

           0       0.59      0.27      0.37      1107
           1       0.83      0.95      0.89      4117

    accuracy                           0.81      5224
   macro avg       0.71      0.61      0.63      5224
weighted avg       0.78      0.81      0.78      5224

Accuracy Score - KNN - Default: 0.8064701378254211


In [27]:
#Parameter tuning with GridSearchCV 

#######################
### K-Nearest Neighbors
#######################
estimator_KNN = KNeighborsClassifier(algorithm='auto')
parameters_KNN = {
    'n_neighbors': (1,10, 1),
    'leaf_size': (20,40,1),
    'p': (1,2),
    'weights': ('uniform', 'distance'),
    'metric': ('minkowski', 'chebyshev'),
}           
# with GridSearch
grid_search_KNN = GridSearchCV(
    estimator=estimator_KNN,
    param_grid=parameters_KNN,
    scoring = 'accuracy',
    n_jobs = -1,
    cv = 5
)
#Documentation of tuneable parameters:
#class sklearn.neighbors.KNeighborsClassifier(n_neighbors=5, weights='uniform', 
#                                             algorithm='auto', leaf_size=30, p=2, 
#                                             metric='minkowski', metric_params=None, 
#                                             n_jobs=None, **kwargs)

In [28]:
KNN_1=grid_search_KNN.fit(X_train, y_train)
y_pred_KNN1 =KNN_1.predict(X_test)
KNN_2=grid_search_KNN.fit(X_train_2, y_train)
y_pred_KNN2 =KNN_2.predict(X_test_2)
KNN_3=grid_search_KNN.fit(X_train_3, y_train)
y_pred_KNN3 =KNN_3.predict(X_test_3)

#Parameter setting that gave the best results on the hold out data.
print(grid_search_KNN.best_params_ ) 
#Mean cross-validated score of the best_estimator
print('Best Score - KNN:', grid_search_KNN.best_score_ )

{'leaf_size': 20, 'metric': 'minkowski', 'n_neighbors': 10, 'p': 1, 'weights': 'distance'}
Best Score - KNN: 0.8197089644809334
